In [35]:
    import abc, jsonschema, rdflib, collections, itertools, operator, copy, functools, pyld.jsonld as jsonld, ast, pathlib, typing
    import strict_rfc3339, rfc3986, rfc3987, IPython, toolz

In [36]:
    class Schema:
        @classmethod
        def schema(cls): return {**collections.ChainMap(*(getattr(object, '__annotations__', {}) or {} for object in cls.__mro__)), 'title': cls.__name__, }
        @classmethod
        def validate(cls, object, schema=None): jsonschema.validate(object, schema or cls.schema(), format_checker=jsonschema.FormatChecker())

In [37]:
    class JsonSchemaMeta(Schema, abc.ABCMeta):
        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = cls.validate(kwargs) or kwargs

    class JsonSchema(JsonSchemaMeta, **jsonschema.Draft7Validator.META_SCHEMA): ...

In [38]:
    def map_container(callable):
        @functools.wraps(callable)
        def logic(object, *args, **kwargs):
            if isinstance(object, (list, dict)): return object.map(callable, *args, **kwargs)
            return object.pipe(callable, *args, **kwargs)
        return logic

In [39]:
class Parser:
    @map_container
    def text(self): return self
    @map_container
    def yaml(self):
        try: from ruamel import yaml
        except: import yaml
        return self.pipe(operator.methodcaller('text'), yaml.safe_load, Type.object)
    @map_container
    def json(self):
        try: import ujson as json
        except: import json
        return self.pipe(operator.methodcaller('text'), json.loads, Type.object)
    @map_container
    def imp(self): return __import__('importlib').import_module(self)
    @map_container
    def git(self): return __import__('git').Repo(self)
    @map_container
    def download_json(self, *args, **object): return Type.object(__import__('requests').get(self, *args, **object).json())

    @map_container
    def download(self, *args, **object): return String.object(__import__('requests').get(self, *args, **object).text)

In [40]:
    class Type(Schema, metaclass=JsonSchema): 
        __annotations__ = {}
        def __new__(cls, object=None, *args, **kwargs):
            if object is None: object = copy.copy(cls.schema().get('default'))
            object = super().__new__(cls, object, *args, **kwargs)
            return cls.validate(object) or object

        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = type(cls).validate(kwargs) or kwargs

        @classmethod
        def object(type, object=None):
            for cls in (type.__subclasses__()):
                try: return cls.object(object)
                except BaseException as e: ...
            else: return type(object)
            
        def pipe(self, *funcs, **kwargs): return toolz.compose(*reversed(funcs))(self, **kwargs)
        def do(self, *funcs, **kwargs): self.pipe(*funcs, **kwargs); return self
        def list(self): return List(self)
        def tuple(self): return tuple(self)
        def set(self): return set(self)
        def str(self): return String.object(self)
        def zip(self, *args): return list(zip(self, *args))
        def starmap(self, callable): return List(itertools.starmap(callable, self))
        
    class Container(Parser):
        def series(self, *args, **kwargs) -> 'pandas.Series': return __import__('pandas').Series(self, *args, **kwargs)
        def frame(self, *args, **kwargs) -> 'pandas.DataFrame': return __import__('pandas').DataFrame(self, *args, **kwargs)
        def __getitem__(self, object): return Type.object(super().__getitem__(object))
        
    class Sequence(Container):
        def map(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.map, toolz.partial(callable, **kwargs)), type(self))
        def filter(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.filter, toolz.partial(callable, **kwargs)), type(self))
        def reduce(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.reduce, toolz.partial(callable, **kwargs)), type(self))
        def groupby(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.groupby, toolz.partial(callable, **kwargs)), Dict)
        def reduceby(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.reduceby, toolz.partial(callable, **kwargs)), Dict)

In [41]:
    class Dict(Type, Container, dict, type='object', default={}): 
        def map(self, value=None, key=None): return Dict({(key or toolz.identity)(k): (value or toolz.identity)(v) for k, v in self.items()})
        def filter(self, value=None, key=None): return Dict({k: v for k, v in self.items() if (
            (key(k) if key else True) and (value(v) if value else True)
        )})
        

In [42]:
    class List(Type, Sequence, list, type='array', default=[]): 
        def __new__(cls, object=None, *args, **kwargs): 
            if isinstance(object, tuple): object = list(object)
            return super().__new__(cls, object, *args, **kwargs)
    class Tuple(Type, Sequence, tuple, type='array', default=tuple()): ...
    class Set(Type, Sequence, set, type='array', uniqueItems=True, default=set()): ...
        

In [ ]:
    class Null(Type, type='null'): 
        def __new__(cls, object=None, *args, **kwargs): return cls.validate(cls, object)
    class Integer(Type, int, type='integer', default=''): ...
    class Number(Type, float, type='number', default=''): ...
    
    class String(Type, Parser, str, type='string', default=''):
        def path(self) -> pathlib.Path: return pathlib.Path(self)
    
    class Date(String, format='date'): ...
    class Datetime(String, format='date-time'): ...
    class Time(String, format='time'): ...
    class Color(String, format='color'): ...
    
    class Email(String, format='email'): ...
    class Uri(String, format='uri'):         
        def text(self, *args, **object): return __import__('requests').get(self,*args, **object).text
        
    class File(String): 
        def text(self): return self.path().read_text()
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().exists(): return 
            raise ValueError(F"{object} is not a file.")
    class Dir(File): 
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().is_dir(): return 
            raise ValueError(F"{object} is not a file.")

In [ ]:
    class StringTypes(ast.NodeTransformer):
        def visit_type(self, node, type=None):
            next = ast.parse(F"""__import__('importlib').import_module('{__name__}').{type}.object('')""").body[0].value
            next.args = [node]
            return ast.copy_location(next, node)
        
        visit_JoinedStr = visit_Str = functools.partialmethod(visit_type, type='String')
        #visit_Dict = functools.partialmethod(visit_type, type='Dict')
        #visit_List = functools.partialmethod(visit_type, type='List')
        #visit_Tuple = functools.partialmethod(visit_type, type='Tuple')

In [ ]:
    def unload_ipython_extension(shell): shell.ast_transformers = [x for x in shell.ast_transformers if not isinstance(x, StringTypes)]
    def load_ipython_extension(shell): unload_ipython_extension(shell) or shell.ast_transformers.append(StringTypes())
    __name__ == '__main__' and load_ipython_extension(get_ipython())

    __name__ == '__main__' and "jschema.ipynb".yaml()['cells'].series().apply("pandas".imp().Series).T

In [ ]:
if __name__ == '__main__':
    __import__('requests_cache').install_cache('jschema')
    "https://api.github.com/users/tonyfast/gists".download_json().frame().T.pipe(display)
    (
        List(range(1, 3))
        .map(str)
        .starmap("https://api.github.com/users/tonyfast/gists?page={}".format)
        .download_json()
        .map("pandas".imp().DataFrame)
        .pipe("pandas".imp().concat)
    ).T.pipe(display)